# Proof of concept: Calculate sun angle
Shadow length and time of day have been collected from http://centrodedescargas.cnig.es. Check
whether these can calculate a hub height that approximately matches the identified hub heights.

In [ ]:
import os
import math

import dotenv
import ephem
import numpy as np
import pandas as pd

dotenv.load_dotenv('../.env')
dotenv.load_dotenv('../.env.secret')

sites = pd.read_csv('../data/poc_measurements.csv')

In [ ]:
sites = sites.assign(
    date_string = lambda x: x.date + ' ' + x.hora + os.environ.get('utc_offset'),
    datetime_local = lambda x: pd.to_datetime(x.date_string, dayfirst=True),
    datetime_utc = lambda x: x.datetime_local.dt.tz_convert("UTC")
)
sites

In [ ]:
site = sites.loc[0]

# https://stackoverflow.com/questions/18157266/how-to-calculate-the-angle-of-the-sun-above-the-horizon-using-pyephem
observer = ephem.Observer()
observer.date = site.datetime_utc
observer.long = site.longitude
observer.lat = site.latitude
sun = ephem.Sun(observer)
sun.compute(observer)
print('Sun angle radians', float(sun.alt))
print('Sun angle degrees', sun.alt * 180 / math.pi)

In [ ]:
observer = ephem.Observer()
for i, site in sites.iloc[:4].iterrows():
    observer.date = site.datetime_utc
    observer.long = site.longitude
    observer.lat = site.latitude
    sun = ephem.Sun(observer)
    sun.compute(observer)
    sites.loc[i, "sun_radians"] = sun.alt

In [ ]:
sites = sites.assign(
    sun_degrees = lambda x: (x.sun_radians * 180 / math.pi).round(1),
    estimated_hub_height=lambda x: np.tan(x.sun_radians) * x.shadow_length
)
sites